## Classification et regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import mstats
from collections import Counter

# Étape 1 Lecture et prétraitement du dataset
def load_and_preprocess_dataset(file_path):
    # Lecture du fichier 
    df = pd.read_csv(file_path, sep=';')
    
    # Suppression des valeurs manquantes
    df.dropna(inplace=True)
    
    # Conversion des colonnes numériques en float
    numeric_columns = ['Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Suppression des nouvelles valeurs manquantes après conversion
    df.dropna(inplace=True)
    
    # Gestion des outliers avec winsorisation
    for col in numeric_columns:
        df[col] = mstats.winsorize(df[col], limits=[0.05, 0.05])  # 5% winsorization

    # Discrétisation des valeurs et remplacement par la moyenne de chaque intervalle
    for col in numeric_columns:
        bins = np.linspace(df[col].min(), df[col].max(), 5)  # 4 intervalles
        labels = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins) - 1)]
        df[col] = pd.cut(df[col], bins=bins, labels=labels).astype(float)
    
    # Normalisation des données numériques
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    
    
    return df

# Étape 2: Calcul de la distance entre deux instances (Manhattan + Hamming)
def calculate_distance(instance1, instance2, categorical_columns, numeric_columns):
    distance = 0.0
    
    # Distance Manhattan pour les colonnes numériques
    for col in numeric_columns:
        distance += abs(instance1[col] - instance2[col])
    
    # Distance de Hamming pour les colonnes catégoriques
    for col in categorical_columns:
        if instance1[col] != instance2[col]:
            distance += 1
    
    return distance

# Étape 3: Trier les instances selon la distance
def sort_by_distance(dataset, target_instance, categorical_columns, numeric_columns):
    distances = []
    for idx, row in dataset.iterrows():
        dist = calculate_distance(row, target_instance, categorical_columns, numeric_columns)
        distances.append((row, dist))
    
    # Trier par distance croissante
    distances.sort(key=lambda x: x[1])
    return [item[0] for item in distances]

# Étape 4: Trouver la classe dominante parmi les k voisins
def get_dominant_class(neighbors, class_column):
    classes = [neighbor[class_column] for neighbor in neighbors]
    return Counter(classes).most_common(1)[0][0]

# Étape 5: Implémentation de l'algorithme k-NN
def knn(dataset, target_instance, k, categorical_columns, numeric_columns, class_column):
    # Retirer 'Label' des colonnes catégoriques
    if class_column in categorical_columns:
        categorical_columns.remove(class_column)
    
    sorted_instances = sort_by_distance(dataset, target_instance, categorical_columns, numeric_columns)
    k_neighbors = sorted_instances[:k]
    return get_dominant_class(k_neighbors, class_column)

# Charger et prétraiter le dataset
file_path = "DatasetExos.csv"  # Remplacez par le chemin réel de votre fichier
df = load_and_preprocess_dataset(file_path)

# Définir les colonnes pour le calcul des distances
categorical_columns = ['ID', 'Label', 'Category', 'Set']
numeric_columns = ['Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z']
class_column = 'Label'

# Instance cible
target_instance = {
    'Acc_x': -0.137, 'Acc_y': 1.066, 'Acc_z': 0.8215,
    'Gyro_x': -6.597, 'Gyro_y': 0.808, 'Gyro_z': 1.985,
    'ID': 'B', 'Category': 'medium', 'Set': 30
}


""" target_instance = {
    'Acc_x': -0.0545,
    'Acc_y': 0.545,
    'Acc_z': 0.896,
    'Gyro_x': -0.8413999999999998,
    'Gyro_y': 0.8538,
    'Gyro_z': 18.778, 
    'ID': 'A',
    'Category': 'heavy',
    'Set': 22
}  """


def normalize_target_instance(target_instance, df, numeric_columns):
    """
    Normalise l'instance cible en utilisant les min et max des colonnes numériques du dataset initial.
    
    Parameters:
        target_instance (dict): Instance cible à normaliser.
        df (DataFrame): Dataset contenant les valeurs min et max des colonnes numériques.
        numeric_columns (list): Liste des colonnes numériques à normaliser.
    
    Returns:
        dict: Instance cible normalisée.
    """
    # Extraire les min et max des colonnes numériques du dataset initial
    min_values = df[numeric_columns].min()
    max_values = df[numeric_columns].max()
    
    # Appliquer la normalisation Min-Max
    for col in numeric_columns:
        target_instance[col] = (target_instance[col] - min_values[col]) / (max_values[col] - min_values[col])
    
    return target_instance


def discretize_target_instance(target_instance, df, numeric_columns):
    for col in numeric_columns:
        bins = np.linspace(df[col].min(), df[col].max(), 5)  # Même nombre d'intervalles
        labels = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins) - 1)]
        target_instance[col] = pd.cut([target_instance[col]], bins=bins, labels=labels).astype(float)[0]
    return target_instance



# Normaliser la target_instance avant de prédire
target_instance = normalize_target_instance(target_instance, df, numeric_columns)
""" # Discrétisation de l'instance cible
target_instance = discretize_target_instance(target_instance, df, numeric_columns) """


# Appliquer k-NN pour K = 3 et K = 10
for k in [3, 10]:
    predicted_class = knn(df, target_instance, k, categorical_columns, numeric_columns, class_column)
    print(f"Classe prédite pour K={k}: {predicted_class}")


Classe prédite pour K=3: bench
Classe prédite pour K=10: bench
